# Decison Support System
### Deciding Factors
- Longevity: time since review
- Length of title and text: count the number of words
- Helpfulness of a review: ratio of “helpful votes” to “total votes”
- Readership: total number of votes of a review
- Select products wtih atleast 100 reviews, eliminated the reviews that had less than 4 votes
- Polarity = positive sentiment + negative sentiment
- Sentiment = positive sentiment - negative sentiment - 2:

In [ ]:
import pandas as pd
import numpy as np
import gzip 
import json
import shutil
import time
import math
import matplotlib.pyplot as plt
import pymc3 as pm
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression

In [ ]:
def getSentimentsAndLengthFromFile(nameOfFile):
    df=pd.DataFrame()
    file = open(nameOfFile, encoding = "utf8")
    count = 0
    column_names = []
    for line in file:
        count=count+1
        if count is 1:
            column_names = line.split('\t')
            df=pd.DataFrame(columns = column_names)
        temp_line = line.rstrip().split('\t')
        df = df.append(pd.Series(temp_line ,index = column_names), ignore_index = True)
    file.close()
    return df

def unzipJson(filename, saveas):
    "both arguments are strings"
    with gzip.open(filename, 'rb') as f_in:
        with open(saveas, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
def getDataFromJsonFile(filename, noOfRev):
    count = 0
    df = pd.DataFrame()
    with open(filename, 'r') as f:
        for line in f:
            if count == noOfRev:
                break
            count = count + 1
            if count%500==0:
                print(count)
            data = json.loads(line)
            dictionary=json.loads(line)
            dictionary["helpful"] = str(dictionary["helpful"][0])+":"+str(dictionary["helpful"][1])
            df = df.append(pd.DataFrame(dictionary,index = [count]), sort = False)
    return df

def plot_traces(trcs, varnames=None):
    '''Plot traces with overlaid means and values'''
    nrows = len(trcs.varnames)
    if varnames is not None:
        nrows = len(varnames)

    ax = pm.traceplot(trcs, varnames=varnames, figsize=(12,nrows*1.4),
                      lines={k: v['mean'] for k, v in
                             pm.summary(trcs,varnames=varnames).iterrows()})

    for i, mn in enumerate(pm.summary(trcs, varnames=varnames)['mean']):
        ax[i,0].annotate('{:.2f}'.format(mn), xy=(mn,0), xycoords='data',
                         xytext=(5,10), textcoords='offset points', rotation=90,
                         va='bottom', fontsize='large', color='#AA0022')

def strip_derived_rvs(rvs):
    '''Remove PyMC3-generated RVs from a list'''
    ret_rvs = []
    for rv in rvs:
        if not (re.search('_log',rv.name) or re.search('_interval',rv.name)):
            ret_rvs.append(rv)
    return ret_rvs

#### Description of the below cell
- unzip the json.gz file 
- get a dataFrame consisting of 35000 reviews
- store this dataFrame in a csv file
- now this csv file is converted to a txt file that is then
    analysed using the sentistrength software, resulting is a
    txt file having analysed result

In [ ]:
# unzipJson("reviews_Electronics_5.json.gz","reviews_Electronics_5.json")
# df = getDataFromJsonFile("reviews_Electronics_5.json",35000)
# df.to_csv("dataset_electronic.csv")


#### Description of the below cell
- dataset_analysed.txt contains all the analysed data that is converted to a dataFrame
- dataFrame is stored in a csv file for faster access next time


In [ ]:
# data = getSentimentsAndLengthFromFile("dataset_analysed.txt")

In [ ]:
# data = data.drop(0,0)
# data.to_csv("dataset.csv")

In [ ]:
# data['polarityText'] = data['reviewTextPositive'] + data['reviewTextNegative']
# data['polaritySummary'] = data['summaryPositive'] + data['summaryNegative\n']
# data['sentimentText'] = data['reviewTextPositive'] - data['reviewTextNegative']-2
# data['sentimentSummary'] = data['summaryPositive'] - data['summaryNegative\n'] - 2
# data['summaryLength'] = data['summary'].str.split().str.len()
# data['reviewTextLength'] = data['reviewText'].str.split().str.len()
# data['readership'] = data['helpful'].str.split(':').str[1]
# data['longetivity'] = time.time() - data['unixReviewTime']
# del data['no']

In [ ]:
# data.to_csv("dataset.csv")

In [ ]:
# read the dataset
data = pd.read_csv('dataset.csv',index_col=0)
data['helpful_votes'] = data['helpful'].str.split(':').str[0]
data['helpful_votes'] = data.helpful_votes.astype(float)
data['longetivity'] = data['longetivity']//86400
# data.head()


In [ ]:
# Select products wtih atleast 100 reviews, eliminated the reviews that had less than 4 votes
tempDf = data.groupby('asin').size().rename("countOfReviews").reset_index()
data = data.merge(tempDf)
data = data[data['countOfReviews'] >= 100]
data = data.loc[data['readership'] >= 4]

In [ ]:
table1Df = pd.DataFrame(columns=['range', 'median', 'mean', 'SD'], 
                        index = ['Rating', 'Longetivity', 'Total Votes', 'Helpful votes', 'Title sentiment',
                                 'Title polarity', 'Review sentiment', 'Review polarity', 'Title length', 'Review length'])

table1Df.loc['Rating'] = pd.Series({
    'range' : str(data['overall'].min())+'-'+str(data['overall'].max()),
    'median' : data['overall'].median(),
    'mean' : data['overall'].mean(),
    'SD' : data['overall'].std()
})
table1Df.loc['Longetivity'] = pd.Series({
    'range' : str(data['longetivity'].min())+'-'+str(data['longetivity'].max()),
    'median' : data['longetivity'].median(),
    'mean' : data['longetivity'].mean(),
    'SD' : data['longetivity'].std()
})
table1Df.loc['Total Votes'] = pd.Series({
    'range' : str(data['readership'].min())+'-'+str(data['readership'].max()),
    'median' : data['readership'].median(),
    'mean' : data['readership'].mean(),
    'SD' : data['readership'].std()
})
table1Df.loc['Helpful votes'] = pd.Series({
    'range' : str(data['helpful_votes'].min())+'-'+str(data['helpful_votes'].max()),
    'median' : data['helpful_votes'].median(),
    'mean' : data['helpful_votes'].mean(),
    'SD' : data['helpful_votes'].std()
})
table1Df.loc['Title sentiment'] = pd.Series({
    'range' : str(data['sentimentSummary'].min())+'-'+str(data['sentimentSummary'].max()),
    'median' : data['sentimentSummary'].median(),
    'mean' : data['sentimentSummary'].mean(),
    'SD' : data['sentimentSummary'].std()
})
table1Df.loc['Title polarity'] = pd.Series({
    'range' : str(data['polaritySummary'].min())+'-'+str(data['polaritySummary'].max()),
    'median' : data['polaritySummary'].median(),
    'mean' : data['polaritySummary'].mean(),
    'SD' : data['polaritySummary'].std()
})
table1Df.loc['Review sentiment'] = pd.Series({
    'range' : str(data['sentimentText'].min())+'-'+str(data['sentimentText'].max()),
    'median' : data['sentimentText'].median(),
    'mean' : data['sentimentText'].mean(),
    'SD' : data['sentimentText'].std()
})
table1Df.loc['Review polarity'] = pd.Series({
    'range' : str(data['polarityText'].min())+'-'+str(data['polarityText'].max()),
    'median' : data['polarityText'].median(),
    'mean' : data['polarityText'].mean(),
    'SD' : data['polarityText'].std()
})
table1Df.loc['Title length'] = pd.Series({
    'range' : str(data['summaryLength'].min())+'-'+str(data['summaryLength'].max()),
    'median' : data['summaryLength'].median(),
    'mean' : data['summaryLength'].mean(),
    'SD' : data['summaryLength'].std()
})
table1Df.loc['Review length'] = pd.Series({
    'range' : str(data['reviewTextLength'].min())+'-'+str(data['reviewTextLength'].max()),
    'median' : data['reviewTextLength'].median(),
    'mean' : data['reviewTextLength'].mean(),
    'SD' : data['reviewTextLength'].std()
})


## Formula
- log(Total Votes) = B0 + B1 * Title_Sentiment + B2 * TITLE_POSTIVE + B3 * Title_length + B4 * Title_Sentiment * TITLE_POSTIVE + B5 * log(Review_Length) + B6 * log(Longevity)

In [ ]:
data
table1Df


In [56]:
readerData = pd.DataFrame()
readerData['Title_Sentiment'] = data['sentimentSummary']
readerData['TITLE_POSITIVE'] = data['polaritySummary']
readerData.loc[readerData['TITLE_POSITIVE'] > 0,'temp'] = 1
readerData.loc[readerData['TITLE_POSITIVE'] <= 0,'temp'] = 0
readerData['TITLE_POSITIVE'] = readerData['temp']
del readerData['temp']
readerData['Title_Length'] = data['summaryLength']
readerData['B4_mul'] = readerData['Title_Sentiment'] * readerData['TITLE_POSITIVE']
readerData['B5_mul'] = np.log(data['reviewTextLength'])
readerData['B6_mul'] = np.log(data['longetivity'])
readerData['TITLE_POSITIVE'] = data['summaryPositive']
logTotalVotes = np.log10(data['readership'])

In [57]:
reg_for_reader = LinearRegression().fit(readerData, logTotalVotes)

In [61]:
readerData['logTotalVotes'] = logTotalVotes
reg_for_reader.coef_
# reg_for_reader.intercept_ 

array([-0.01047036, -0.07413579,  0.0048672 ,  0.04432912,  0.09628507,
        0.42113113])

In [71]:
helpful = pd.DataFrame()
helpful['Review_Sentiment'] = data['sentimentText']
helpful['REVIEW_NEUTRAL'] = data['polarityText']
helpful.loc[helpful['REVIEW_NEUTRAL'] == 0,'temp'] = 1
helpful.loc[helpful['REVIEW_NEUTRAL'] != 0,'temp'] = 0
helpful['REVIEW_NEUTRAL'] = helpful
['temp']
del helpful['temp']
helpful['B3_mul'] = np.log(data['reviewTextLength'])
helpful['B4_mul'] = helpful['Review_Sentiment']*helpful['REVIEW_NEUTRAL']
helpful['B5_mul'] = np.log(data['longetivity'])
numerator = data['helpful'].str.split(':',n=1,expand=True)[0].astype(float)
denominator = data['helpful'].str.split(':',n=1,expand=True)[1].astype(float)
ratio=numerator/denominator

In [72]:
reg_for_helpful = LinearRegression().fit(helpful, ratio)

In [81]:
helpful['ratio'] = ratio
reg_for_helpful.coef_
print("This is for readership:",reg_for_reader.coef_)
print("This is for helpfulness:",reg_for_helpful.coef_)


This is for readership: [-0.01047036 -0.07413579  0.0048672   0.04432912  0.09628507  0.42113113]
This is for helpfulness: [ 0.02935702  0.02935702  0.09910268 -0.00994863  0.19413566]


### Summary of Hypothesis testing

In [79]:
table2Df = pd.DataFrame(columns=['Hypothesized relationship', 'Estimates (Wald Chi-square)', 'Results'], 
                        index = ['H1','H2','H3','H4','H5','H6','H7','H8'])
table2Df.loc['H1'] = pd.Series({
    'Hypothesized relationship' : 'Longevity → readership',
    'Estimates (Wald Chi-square)': reg_for_reader.coef_[5],
    'Results':'lol'
})
table2Df.loc['H2'] = pd.Series({
    'Hypothesized relationship' : 'Title sentiment → readership',
    'Estimates (Wald Chi-square)': reg_for_reader.coef_[0],
    'Results':'lol'
})
table2Df.loc['H3'] = pd.Series({
    'Hypothesized relationship' : 'Title sentiment × title positive → readership',
    'Estimates (Wald Chi-square)': reg_for_reader.coef_[3],
    'Results':'lol'
})
table2Df.loc['H4'] = pd.Series({
    'Hypothesized relationship' : 'Title length → readership',
    'Estimates (Wald Chi-square)': reg_for_reader.coef_[4],
    'Results':'lol'
})
table2Df.loc['H5'] = pd.Series({
    'Hypothesized relationship' : 'Review length → readership',
    'Estimates (Wald Chi-square)': 'lol',
    'Results':'lol'
})
table2Df.loc['H6'] = pd.Series({
    'Hypothesized relationship' : 'Review length → helpfulness',
    'Estimates (Wald Chi-square)': 'lol',
    'Results':'lol'
})
table2Df.loc['H7'] = pd.Series({
    'Hypothesized relationship' : 'Review sentiment → helpfulness',
    'Estimates (Wald Chi-square)': 'lol',
    'Results':'lol'
})
table2Df.loc['H8'] = pd.Series({
    'Hypothesized relationship' : 'Review sentiment × review neutral → helpfulness',
    'Estimates (Wald Chi-square)': 'lol',
    'Results':'lol'
})

print("This is for readership:",reg_for_reader.coef_)
print("This is for helpfulness:",reg_for_helpful.coef_)

table2Df

This is for readership: [-0.01047036 -0.07413579  0.0048672   0.04432912  0.09628507  0.42113113]
This is for helpfulness: [ 0.02935702  0.02935702  0.09910268 -0.00994863  0.19413566]


,Hypothesized relationship,Estimates (Wald Chi-square),Results
H1,Longevity → readership,lol,lol
H2,Title sentiment → readership,lol,lol
H3,Title sentiment × title positive → readership,lol,lol
H4,Title length → readership,lol,lol
H5,Review length → readership,lol,lol
H6,Review length → helpfulness,lol,lol
H7,Review sentiment → helpfulness,lol,lol
H8,Review sentiment × review neutral → helpfulness,lol,lol


### Graphs
#### The below contains 4 graphs as in the paper 

- polartiyText : ReviewPolarity
- sentimentText : ReviewSentiment
- polaritySummmary : TitlePolarity
- Title_Sentiment : TitleSentiment

In [ ]:
g = sns.catplot(x='polarityText',data=data, kind='count', aspect=1.5)
g = sns.catplot(x='sentimentText',data=data, kind='count', aspect=1.5)
g = sns.catplot(x='polaritySummary',data=data, kind='count', aspect=1.5)
g = sns.catplot(x='Title_Sentiment',data=readerData, kind='count', aspect=1.5)